In [ ]:
import pandas as pd
import geopandas as gpd
import os
import glob
import random
import time

# ruta base donde estan las carpetas que contienen los csvs de https://github.com/LMCappelletti/data.git

directorio_dataFundar = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data'


# dict para guardar los nombres de todas las columnas de todos los csvs de
# todas las carpeta dataFundar
dict_ALLcsv_ALLcolumnas = {}

# recorrer todas las carpetas y subcarpetas
# recorre todos los directorios y subdirectorios desde la ruta base, obteniendo archivos en cada carpeta

# root: la ruta actual del directorio que se está recorriendo (la carpeta con todos los topicos)
# dirs: lista con los nombres de las subcarpetas que se encuentran en la carpeta actual
# files: lista con los nombres de los archivos en la carpeta actual

for root, dirs, files in os.walk(directorio_dataFundar): 
    
    # recorre todos las carpetas y subcarpetas desde la ruta base, obteniendo archivos en cada carpeta
    for file in files:
        file_dir = os.path.join(root, file)
        
        # solo considerar archivos con extensión .csv
        if file.endswith('.csv'):
            # solo leemos las columnas, estoy teniendo en cuenta que estos .csvs
            # segun la metadata del repositorio en la fila 0 presenta los
            # titulos de cada columna
            df = pd.read_csv(file_dir, nrows = 0, encoding = 'ISO-8859-1')
            # para resolver el error este que me aparecio
            # de la codificacion de caracteres: 
            # utf-8' codec can't decode byte 0xe1 in position 217: invalid
            # continuation byte 
            dict_ALLcsv_ALLcolumnas[file] = list(df.columns)


In [3]:
print(dict_ALLcsv_ALLcolumnas)

{'geonomenclador_export_input.csv': ['geocodigo', 'name_long', 'name_short', 'iso_2'], 'ISA_composicion-ipcf_it4.csv': ['year', 'semestre', 'genero', 'proporcion'], 'ISA_tipo_empleo_i1.csv': ['ano', 'variable', 'valor'], 'ISA_salarios_region_i2.csv': ['ano_trim', 'variable', 'valor'], 'ISA_salario_real_i1.csv': ['ano', 'variable', 'valor'], 'ISA_ipcf_it1.csv': ['year', 'semestre', 'ipcf_index'], 'ISA_regiones-ipcf_it1.csv': ['year', 'semestre', 'region', 'deflactor', 'indice'], 'ISA_salario_real_i3.csv': ['ano', 'salario'], 'salario_real_ppa2017_ceped.csv': ['iso3', 'anio', 'salario_real_ppa_consumo_privado_2017'], 'ISA_composicion-ipcf_it3.csv': ['year', 'semestre', 'edad_jefe', 'fuente', 'proporcion'], 'ISA_salarios_mundo_i2.csv': ['ano', 'variable', 'valor'], 'ISA_salarios_mundo_i1.csv': ['pais', 'salariohorario'], 'ISA_salarios_region_i1.csv': ['region', 'variable', 'valor'], 'ISA_composicion-ipcf_it5.csv': ['year', 'semestre', 'fuente', 'indice'], 'ISA_regiones-ipcf_it2.csv': ['ye

In [8]:
len(dict_ALLcsv_ALLcolumnas)

338